In [1]:
%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd

import gc
import timeit
import sys
import os

from random import normalvariate
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# Prepare the synthetic observation vector for all municipalities

In [3]:
def make_synt_obs(municipality, df_top_hits, c_v, semi_average):
    save_dir = "synthetic_data/"
    df_municipality = pd.read_pickle(save_dir + municipality)
    pop = np.zeros((20, 15))
    for i in range(20):
        parameter = df_top_hits.iloc[i, 0:9]
        df_hits = df_municipality.loc[
            (df_municipality["init_pop"] == parameter[0])
            & (df_municipality["init_hps"] == parameter[1])
            & (df_municipality["sh_threshold"] == parameter[2])
            & (df_municipality["i_fcalves"] == parameter[3])
            & (df_municipality["i_yhinds"] == parameter[4])
            & (df_municipality["i_ahinds"] == parameter[5])
            & (df_municipality["i_mcalves"] == parameter[6])
            & (df_municipality["i_ystags"] == parameter[7])
            & (df_municipality["i_astags"] == parameter[8])
        ].copy()
        pop[i, :] = df_hits.iloc[:, 17].values

    pop_bh = np.median(pop, axis=0)

    # Ensure that the sampling gives a correct c_v and semi_average value
    CV = 0
    SA = 0
    while (abs(c_v - CV) > 0.01) & (abs(semi_average - SA) > 0.1 * semi_average):
        synt_obs = np.zeros(15)
        for i in range(15):
            synt_obs[i] = normalvariate(pop_bh[i], pop_bh[i] * c_v)

        CV = np.std(synt_obs) / np.mean(synt_obs)
        SA = synt_obs[8:15].mean() / synt_obs[0:7].mean()

    # Scale synt_obs around 1.0
    synt_obs_scaled = synt_obs / np.mean(synt_obs)
    return synt_obs_scaled

# Residual Sum of Squares calculations: synthetic data against data for seen deer per hunting hour

In [4]:
# Revoking stored reported data
# https://ipython.org/ipython-doc/rel-0.12/config/extensions/storemagic.html

%store -r data_Averoy
%store -r data_Tingvoll
%store -r data_Surnadal
%store -r data_Sunndal
%store -r data_Vestnes
%store -r data_Laerdal

In [5]:
def sum_squares_seen_deer(
    number_of_years,
    municipality_file,
    empirical_observations,
    compare,
):
    """
    Calculating the minimum residual_sum_of_squares between synthetic data and
    reported data
    """

    # Revoking the targeted municipality data frame created by the synthetic population generator
    save_dir = "synthetic_data/"
    data_frame = pd.read_pickle(save_dir + municipality_file)

    # Defining column to extract values from
    if compare == "before_hunting":
        comp_choice = "tot_pop_bh"
    if compare == "after_hunting":
        comp_choice = "tot_pop_ah"

    RSS_list = []
    for i in range(0, int(len(data_frame) / number_of_years)):
        # Catch the predicted total population sizes for all observation years
        seen_deer_p = data_frame.iloc[i * number_of_years : (i + 1) * number_of_years][
            comp_choice
        ].values

        seen_deer_e = empirical_observations

        # Find the scaling factor that minimises RSS and do the scaling
        scaling_factor = np.sum(np.multiply(seen_deer_p, seen_deer_e)) / np.sum(
            np.multiply(seen_deer_p, seen_deer_p)
        )
        seen_deer_p_scaled = scaling_factor * seen_deer_p

        # Find minimum Residual Sum Square value
        RSS_min = np.sum((seen_deer_p_scaled - seen_deer_e) ** 2)

        RSS_list.append([scaling_factor, RSS_min])

    return RSS_list

In [6]:
def make_RSS_frame(municipality_file, RSS_list):

    # Revoking the targeted municipality data frame created by the synthetic population generator
    save_dir = "synthetic_data/"
    data_frame = pd.read_pickle(save_dir + municipality_file)

    # Remove all rows except those where obs_year == 2021
    df_filtered = data_frame[data_frame["obs_year"] == 2021]

    # Remove columns not used
    cols = [
        "f_calves",
        "y_hinds",
        "a_hinds",
        "m_calves",
        "y_stags",
        "a_stags",
        "ws_fc",
        "ws_yh",
        "ws_ah",
        "ws_mc",
        "ws_ys",
        "ws_as",
        "c_yh",
        "c_ah",
    ]
    df_filtered2 = df_filtered.drop(cols, axis=1)

    # Resert index, otherwise pd.concat does not work
    df_filtered2 = df_filtered2.reset_index(drop=True)

    # Add two columns from RSS_list
    df_RSS = pd.concat(
        [df_filtered2, pd.DataFrame(RSS_list, columns=["scaling", "RSS"])], axis=1
    )

    # Sort the frame on the RSS value
    sorted_sum_squares_frame = df_RSS.sort_values(by=["RSS"]).reset_index(drop=True)

    return sorted_sum_squares_frame

In [7]:
def extract_top_hits(
    i,
    sorted_sum_squares_frame,
    filtering_strategy,
    frac_init_pop,
    number_of_top_hits,
):
    """
    Filtering the sum_squares frame based on an assumption about the
    size of the Dec 31 2020 population vs the Dec 31 2006 population,
    and delivering only the number_of_top_hits best fits.
    """

    if filtering_strategy == "uninformed":
        sorted_sum_squares_frame_filtered = sorted_sum_squares_frame

    if filtering_strategy == "informed":
        # Educated guess filtering - can play with these criteria
        sorted_sum_squares_frame_filtered = sorted_sum_squares_frame[
            (
                sorted_sum_squares_frame.tot_pop_ah
                > frac_init_pop[i] * sorted_sum_squares_frame.init_pop
            )
        ].reset_index(drop=True)

    return sorted_sum_squares_frame_filtered[0:number_of_top_hits]

In [8]:
def run_the_show():
    global tot_pop_ah_collected
    tot_pop_ah_collected = []
    save_dir = "synthetic_data/"

    # Coefficient of variation of the original seen-deer vectors + semi-average values
    # Data calculated in Figure1_..
    c_v = [0.218, 0.110, 0.308, 0.151, 0.164, 0.179]
    semi_average_seen_deer = [0.838, 1.008, 0.864, 1.011, 0.942, 1.041]

    # Using all 6 municipalities
    municipalities = ["Averoy", "Tingvoll", "Surnadal", "Sunndal", "Vestnes", "Laerdal"]
    data_municipality = [
        data_Averoy,
        data_Tingvoll,
        data_Surnadal,
        data_Sunndal,
        data_Vestnes,
        data_Laerdal,
    ]

    top_hits_Averoy_seen_deer = pd.read_pickle(
        save_dir + "top_hits_Averoy_seen_deer.pkl"
    )
    top_hits_Tingvoll_seen_deer = pd.read_pickle(
        save_dir + "top_hits_Tingvoll_seen_deer.pkl"
    )
    top_hits_Surnadal_seen_deer = pd.read_pickle(
        save_dir + "top_hits_Surnadal_seen_deer.pkl"
    )
    top_hits_Sunndal_seen_deer = pd.read_pickle(
        save_dir + "top_hits_Sunndal_seen_deer.pkl"
    )
    top_hits_Vestnes_seen_deer = pd.read_pickle(
        save_dir + "top_hits_Vestnes_seen_deer.pkl"
    )
    top_hits_Laerdal_seen_deer = pd.read_pickle(
        save_dir + "top_hits_Laerdal_seen_deer.pkl"
    )

    municipality_frame = [
        "df_original_sorted_Averoy.pkl",
        "df_original_sorted_Tingvoll.pkl",
        "df_original_sorted_Surnadal.pkl",
        "df_original_sorted_Sunndal.pkl",
        "df_original_sorted_Vestnes.pkl",
        "df_original_sorted_Laerdal.pkl",
    ]

    top_hits_seen_deer = [
        top_hits_Averoy_seen_deer,
        top_hits_Tingvoll_seen_deer,
        top_hits_Surnadal_seen_deer,
        top_hits_Sunndal_seen_deer,
        top_hits_Vestnes_seen_deer,
        top_hits_Laerdal_seen_deer,
    ]

    compare = "before_hunting"
    filtering_strategy = "informed"
    frac_init_pop = [0.8] * len(municipalities)  # minumum tot_pop 2020/2006 ratio
    number_of_top_hits = 20

    for k in range(10):
        print("k = ", k)
        tot_pop_ah = []
        for q in range(len(municipalities)):
            [
                municipality,
                first_year,
                last_year,
                number_of_years,
                years,
                seen_deer_obs,
                seen_deer_obs_outfield,
                seen_deer_obs_infield,
                hinds_per_stag_obs,
                total_harvest,
                fraction_female_calves_harvested,
                fraction_young_hinds_harvested,
                fraction_adult_hinds_harvested,
                fraction_male_calves_harvested,
                fraction_young_stags_harvested,
                fraction_adult_stags_harvested,
                spring_counts,
            ] = data_municipality[q]

            # Make synthetic observation record for the given municipality
            synt_obs_scaled = make_synt_obs(
                municipality_frame[q],
                top_hits_seen_deer[q],
                c_v[q],
                semi_average_seen_deer[q],
            )

            RSS_list = sum_squares_seen_deer(
                number_of_years,
                municipality_frame[q],
                synt_obs_scaled,
                compare,
            )

            sorted_sum_squares_frame = make_RSS_frame(municipality_frame[q], RSS_list)

            top_hits_frame_filtered = extract_top_hits(
                q,
                sorted_sum_squares_frame,
                filtering_strategy,
                frac_init_pop,
                number_of_top_hits,
            )

            # Extract post-hunt median population size
            tot_pop_ah.append(np.median(top_hits_frame_filtered["tot_pop_ah"].values))

        tot_pop_ah_collected.append(tot_pop_ah)
    return

In [9]:
%%time
# Running all scripts by calling up run_the_show
# This is done to get rid of memory leaks
run_the_show()

k =  0
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
CPU times: user 1d 16h 12min 16s, sys: 3h 15min 29s, total: 1d 19h 27min 45s
Wall time: 1d 19h 33min 21s


In [10]:
np.array(tot_pop_ah_collected)

array([[1022. , 2505. , 1445. , 2221.5, 1791.5, 1560.5],
       [1039.5, 2118.5, 1513. , 1592. , 2240.5, 1483. ],
       [1027. , 2237. , 1264.5, 1630.5, 1804.5, 1363.5],
       [1129.5, 2214.5, 1323.5, 2185. , 1850. , 1686. ],
       [1013. , 2242.5, 1521. , 1680. , 2061.5, 1843. ],
       [1013. , 2660. , 2496. , 1900.5, 2089.5, 1460. ],
       [1020. , 2298.5, 1574. , 1823.5, 2465. , 1306. ],
       [1099.5, 1988. , 1490. , 1584.5, 2264. , 1395. ],
       [1032. , 2200.5, 2135.5, 1942. , 2185. , 1912. ],
       [1051.5, 2103. , 1534. , 2272.5, 1620.5, 1970. ]])